In [21]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os, re, sys, time

# sklearn stuff
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.externals import joblib

# keras stuff
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Conv2D, MaxPooling2D, Flatten
from keras.optimizers import SGD
from keras.metrics import binary_accuracy
from keras import callbacks

# scipy stuff
from scipy.interpolate import interp1d
from scipy.stats import mode

# preprocessing stuff
from preprocessingTR import *

%matplotlib inline

In [2]:
def load_data(xpath='./Data_test/processed/full.pickle'):
    return pd.read_pickle(xpath)

def scale_data(df, model_dir='./Models/'):
    """
    Scale data
    """
    xTe = df.values
    ss = joblib.load(model_dir + 'ss.pkl')
    xTe = ss.transform(xTe)

    return xTe

In [5]:
# generate user dictionary
u_dict = generate_udict('./Data_test/')

# preprocess and save training data
preprocess_and_save(u_dict, dir_name='./Data_test/processed/')

# merge and save files as binary objects for quick loading
merge_incremental(base='./Data_test/processed/')

# load and split data
data = load_data()
xTe = scale_data(data)

Files for user 0 exist on disk.
Nothing to merge


In [ ]:
from preprocessingTR import *

In [ ]:
print('Loading model')
model = keras.models.load_model('./Models/ConvNetC.h5')

In [ ]:
preds = model.predict(xTe_conv)

In [ ]:
user_lookup = {0: 'other', 1: 'Mukund', 2: 'Frank'}

In [ ]:
id_predicted = mode(preds.round().argmax(axis=1)).mode[0]

In [ ]:
if id_predicted == 0:
    print('Get the fuck out of here!')
else:
    print('Welcome back %s!' % user_lookup[id_predicted])

In [5]:
with open('./Data_test/RESULT', 'w') as f:
    f.writelines('hello')